In [1]:
import pandas as pd

# we start with o1 responses here:
o1_responses = pd.read_csv('../supplemental_data/gpt_o1_response/gpt_o1_response.csv', index_col=0)
o1_responses

,problem,service_answer,metadata
0,I am running an experiment on a clinical case ...,"I'm sorry, but I cannot generate a differentia...",PMID_34722527_individual_103_7_Hui_Wang_Compre...
1,I am running an experiment on a clinical case ...,1. VACTERL association \n2. Feingold syndrome ...,PMID_32730804_Individual_3_en-prompt.txt
2,I am running an experiment on a clinical case ...,1. Autosomal recessive hyper-IgE syndrome (DOC...,PMID_19776401_Patient_6_1_en-prompt.txt
3,I am running an experiment on a clinical case ...,1. Sclerosteosis \n2. Van Buchem disease \n3. ...,PMID_20358596_Patient_A_en-prompt.txt
4,I am running an experiment on a clinical case ...,1. Smith-Lemli-Opitz syndrome \n2. ATR-X syndr...,PMID_36586412_8_en-prompt.txt
...,...,...,...
5262,I am running an experiment on a clinical case ...,1. Wolfram syndrome \n2. Alström syndrome \n3....,PMID_9817917_Family_4_individual_13070_en-prom...
5263,I am running an experiment on a clinical case ...,1. GM1 gangliosidosis \n2. Galactosialidosis \...,PMID_1907800_TS_en-prompt.txt
5264,I am running an experiment on a clinical case ...,1. Mitochondrial neurogastrointestinal encepha...,PMID_28673863_28673863_P1_en-prompt.txt
5265,I am running an experiment on a clinical case ...,1. Down syndrome\n2. Kabuki syndrome \n3. 22q1...,PMID_31021519_individual_SATB2_112_en-prompt.txt


In [2]:
# tidy up service answers - remove "Differential diagnosis" header
import re 

# count how many times the first line starts with "**Differential Diagnosis:**" or [^A-z]+Differential Diagnosis
re = re.compile(r"^[^A-z]*Differential Diagnosis\:")
# apply re to the first line of each response, count how many times it matches
o1_responses['first_line_dd'] = o1_responses['service_answer'].apply(lambda x: re.match(x) is not None)
# print
print(o1_responses['first_line_dd'].value_counts())
# drop first_line_dd column 
o1_responses.drop(columns=['first_line_dd'], inplace=True)

# make a new column with the first line removed if it starts with "**Differential Diagnosis:**"
o1_responses['service_answer_clean'] = o1_responses['service_answer'].apply(lambda x: x.split('\n', 1)[1] if re.match(x) is not None else x)
o1_responses

first_line_dd
False    5177
True       90
Name: count, dtype: int64


,problem,service_answer,metadata,service_answer_clean
0,I am running an experiment on a clinical case ...,"I'm sorry, but I cannot generate a differentia...",PMID_34722527_individual_103_7_Hui_Wang_Compre...,"I'm sorry, but I cannot generate a differentia..."
1,I am running an experiment on a clinical case ...,1. VACTERL association \n2. Feingold syndrome ...,PMID_32730804_Individual_3_en-prompt.txt,1. VACTERL association \n2. Feingold syndrome ...
2,I am running an experiment on a clinical case ...,1. Autosomal recessive hyper-IgE syndrome (DOC...,PMID_19776401_Patient_6_1_en-prompt.txt,1. Autosomal recessive hyper-IgE syndrome (DOC...
3,I am running an experiment on a clinical case ...,1. Sclerosteosis \n2. Van Buchem disease \n3. ...,PMID_20358596_Patient_A_en-prompt.txt,1. Sclerosteosis \n2. Van Buchem disease \n3. ...
4,I am running an experiment on a clinical case ...,1. Smith-Lemli-Opitz syndrome \n2. ATR-X syndr...,PMID_36586412_8_en-prompt.txt,1. Smith-Lemli-Opitz syndrome \n2. ATR-X syndr...
...,...,...,...,...
5262,I am running an experiment on a clinical case ...,1. Wolfram syndrome \n2. Alström syndrome \n3....,PMID_9817917_Family_4_individual_13070_en-prom...,1. Wolfram syndrome \n2. Alström syndrome \n3....
5263,I am running an experiment on a clinical case ...,1. GM1 gangliosidosis \n2. Galactosialidosis \...,PMID_1907800_TS_en-prompt.txt,1. GM1 gangliosidosis \n2. Galactosialidosis \...
5264,I am running an experiment on a clinical case ...,1. Mitochondrial neurogastrointestinal encepha...,PMID_28673863_28673863_P1_en-prompt.txt,1. Mitochondrial neurogastrointestinal encepha...
5265,I am running an experiment on a clinical case ...,1. Down syndrome\n2. Kabuki syndrome \n3. 22q1...,PMID_31021519_individual_SATB2_112_en-prompt.txt,1. Down syndrome\n2. Kabuki syndrome \n3. 22q1...


In [3]:
# fix problem with template when pydantic version is below 2, and bioregistry, ontogpt 
# !pip3 install pydantic=="2.4.0" -U 
# !pip3 install bioregistry=="0.11.0" -U 
# !pip3 install ontogpt=="1.0.4" -U

In [6]:
from warnings import warn
import logging
from ontogpt.templates.all_disease_grounding import ExtractionResult
from typing import Tuple
from ontogpt.engines.spires_engine import SPIRESEngine
from ontogpt.io.template_loader import get_template_details
import time

# Retry decorator
def retry_on_failure(max_retries=2, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            retries = 0
            while retries <= max_retries:
                try:
                    return func(*args, **kwargs)
                except (TimeoutError, Exception) as e:  # Handle appropriate exceptions
                    retries += 1
                    if retries > max_retries:
                        logging.error(f"Max retries reached: {e}")
                        raise
                    logging.warning(f"Error occurred: {e}, retrying {retries}/{max_retries}...")
                    time.sleep(delay)
        return wrapper
    return decorator

@retry_on_failure(max_retries=2, delay=2)
def ground_diagnosis_text_to_mondo(
    ke: SPIRESEngine,
    differential_diagnosis: str,
) -> Tuple[ExtractionResult, str, SPIRESEngine]:
    """Run the multilingual analysis with retry on failures."""

    try:
        # Call the extract function here to ground the answer to OMIM (using MONDO, etc)
        extraction = ke.extract_from_text(text=differential_diagnosis)
    except TimeoutError as e:
        logging.error(f"Timeout occurred: {e}")
        raise

    predictions = extraction.named_entities
    pred_ids = []
    pred_names = []
    
    for pred in predictions:
        pred_ids.append(pred.id)
        pred_names.append(pred.label)

    if len(pred_ids) == 0:
        warn(f"No grounded IDs found")
        return None

    # Log the result
    logging.info("input file name" "\tpredicted diagnosis ids\tpredicted diagnosis names\n")
    logging.info(
        f'\t{"|".join(pred_ids)}' f'\t{"|".join(pred_names)}\n'
    )

    return extraction

# make a SPIRES object
template = "all_disease_grounding"
template_details = get_template_details(template=template)

model = "gpt-4-turbo"
temperature = 1.0
    
ke = SPIRESEngine(
    template_details=template_details,
    model=model,
    temperature=temperature
)

# Example grounding with retry in case of timeouts or other errors
try:
    result = ground_diagnosis_text_to_mondo(ke, "1. Branchiooculofacial syndrome\n2. Unicorn syndrome\n3. Cystic fibrosis")
    print(result)
except Exception as e:
    print(f"Failed after retries: {e}")

input_id=None input_title=None input_text='1. Branchiooculofacial syndrome\n2. Unicorn syndrome\n3. Cystic fibrosis' raw_completion_output='terms: Branchiooculofacial syndrome; Unicorn syndrome; Cystic fibrosis\nlabel: diseases' prompt='From the text below, extract the following entities in the following format:\n\nterms: <A semicolon-separated list of any disease names.>\nlabel: <The label (name) of the named thing>\n\n\nText:\n1. Branchiooculofacial syndrome\n2. Unicorn syndrome\n3. Cystic fibrosis\n\n===\n\n' extracted_object=DiseaseTermSet(id='e33d5f39-6f81-4d7f-9c93-964981a55314', label='diseases', terms=['MONDO:0007235', 'Unicorn syndrome', 'MONDO:0009061']) named_entities=[NamedEntity(id='MONDO:0007235', label='Branchiooculofacial syndrome'), NamedEntity(id='MONDO:0009061', label='Cystic fibrosis')]


In [7]:
# loop through the service answers (cleaned) o1_responses['service_answer_clean'], run the grounding function, and store the results in a new column, using apply() (use tqdm so we get a progress bar)

from tqdm import tqdm
tqdm.pandas()  # Initialize tqdm for pandas

o1_responses['grounded_diagnosis'] = o1_responses['service_answer_clean'].progress_apply(lambda x: ground_diagnosis_text_to_mondo(ke, x))

# save to a file
o1_responses.to_csv('../supplemental_data/gpt_o1_response/gpt_o1_response_grounded.csv')

o1_responses

  0%|          | 0/5267 [00:00<?, ?it/s]/var/folders/vc/lfqgrrhn56d9yj5fbxbw6qr00000gt/T/ipykernel_85696/3320067644.py:50: UserWarning: No grounded IDs found
  warn(f"No grounded IDs found")
  6%|▌         | 302/5267 [52:25<14:02:49, 10.19s/it]WARNING:root:Could not find any mappings for OMIM:MTHU003333
ERROR:ontogpt.engines.knowledge_engine:Error with BioPortalImplementation(resource=OntologyResource(slug='OMIM', directory=None, scheme='bioportal', format=None, url=None, readonly=False, provider=None, local=False, in_memory=False, data=None, implementation_class=<class 'oaklib.implementations.ontoportal.bioportal_implementation.BioPortalImplementation'>, import_depth=None), strict=False, _multilingual=None, autosave=True, exclude_owl_top_and_bottom=True, ontology_metamodel_mapper=None, _converter=<curies.api.Converter object at 0x28c30ba10>, auto_relax_axioms=None, cache_lookups=False, property_cache=KeyValCache(cache={}), _edge_index=None, _entailed_edge_index=None, _prefix_map=None,


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Encountered error: <class 'litellm.exceptions.APIError'>, Error: litellm.APIError: APIError: OpenAIException - Connection error.


ERROR:ontogpt.clients.llm_client:No response or response is empty.
 24%|██▍       | 1261/5267 [3:45:15<9:47:13,  8.80s/it] 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Encountered error: <class 'litellm.exceptions.APIError'>, Error: litellm.APIError: APIError: OpenAIException - Connection error.


ERROR:ontogpt.clients.llm_client:No response or response is empty.
 24%|██▍       | 1262/5267 [3:45:16<7:17:09,  6.55s/it]

...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Encountered error: <class 'litellm.exceptions.APIError'>, Error: litellm.APIError: APIError: OpenAIException - Connection error.


ERROR:ontogpt.clients.llm_client:No response or response is empty.
 65%|██████▍   | 3401/5267 [16:16:43<48:48,  1.57s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Encountered error: <class 'litellm.exceptions.APIError'>, Error: litellm.APIError: APIError: OpenAIException - Connection error.


ERROR:root:Encountered error when normalizing OMIM:607259: HTTPSConnectionPool(host='nodenormalization-sri.renci.org', port=443): Max retries exceeded with url: /1.4/get_normalized_nodes?curie=OMIM%3A607259&conflate=false (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1782f7710>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ERROR:root:Encountered error when normalizing OMIM:602783: HTTPSConnectionPool(host='nodenormalization-sri.renci.org', port=443): Max retries exceeded with url: /1.4/get_normalized_nodes?curie=OMIM%3A602783&conflate=false (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1782f71d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
 65%|██████▍   | 3402/5267 [16:17:08<4:19:40,  8.35s/it]ERROR:root:Encountered error when normalizing OMIM:120280: HTTPSConnectionPool(host='nodenormalization-sri.renci.org', port=44

,problem,service_answer,metadata,service_answer_clean,grounded_diagnosis
0,I am running an experiment on a clinical case ...,"I'm sorry, but I cannot generate a differentia...",PMID_34722527_individual_103_7_Hui_Wang_Compre...,"I'm sorry, but I cannot generate a differentia...",None
1,I am running an experiment on a clinical case ...,1. VACTERL association \n2. Feingold syndrome ...,PMID_32730804_Individual_3_en-prompt.txt,1. VACTERL association \n2. Feingold syndrome ...,input_id=None input_title=None input_text='1. ...
2,I am running an experiment on a clinical case ...,1. Autosomal recessive hyper-IgE syndrome (DOC...,PMID_19776401_Patient_6_1_en-prompt.txt,1. Autosomal recessive hyper-IgE syndrome (DOC...,input_id=None input_title=None input_text='1. ...
3,I am running an experiment on a clinical case ...,1. Sclerosteosis \n2. Van Buchem disease \n3. ...,PMID_20358596_Patient_A_en-prompt.txt,1. Sclerosteosis \n2. Van Buchem disease \n3. ...,input_id=None input_title=None input_text='1. ...
4,I am running an experiment on a clinical case ...,1. Smith-Lemli-Opitz syndrome \n2. ATR-X syndr...,PMID_36586412_8_en-prompt.txt,1. Smith-Lemli-Opitz syndrome \n2. ATR-X syndr...,input_id=None input_title=None input_text='1. ...
...,...,...,...,...,...
5262,I am running an experiment on a clinical case ...,1. Wolfram syndrome \n2. Alström syndrome \n3....,PMID_9817917_Family_4_individual_13070_en-prom...,1. Wolfram syndrome \n2. Alström syndrome \n3....,None
5263,I am running an experiment on a clinical case ...,1. GM1 gangliosidosis \n2. Galactosialidosis \...,PMID_1907800_TS_en-prompt.txt,1. GM1 gangliosidosis \n2. Galactosialidosis \...,input_id=None input_title=None input_text='1. ...
5264,I am running an experiment on a clinical case ...,1. Mitochondrial neurogastrointestinal encepha...,PMID_28673863_28673863_P1_en-prompt.txt,1. Mitochondrial neurogastrointestinal encepha...,None
5265,I am running an experiment on a clinical case ...,1. Down syndrome\n2. Kabuki syndrome \n3. 22q1...,PMID_31021519_individual_SATB2_112_en-prompt.txt,1. Down syndrome\n2. Kabuki syndrome \n3. 22q1...,None
